In [0]:
import numpy as np0
import jax
import jax.numpy as np
import jax.random as random
import grad_constr_jax as gc
import mectools.endy as nd

## Test Model

In [0]:
N = 100

In [0]:
def con(x, y):
    return [
        np.sum(x*x) - 1.0,
        np.sum(y*y) - 1.0
    ]
def obj(x, y):
    return -(np.sum(x)+np.sum(y))

In [0]:
key = random.PRNGKey(0)
var0 = {
    'x': random.uniform(key, (N,)),
    'y': random.uniform(key, (N,))
}

In [0]:
%time var1 = gc.constrained_gradient_descent(obj, con, var0, output=True)

In [0]:
np.max(np.abs(var1['x']-1/np.sqrt(N)))

## Macro Model

In [0]:
N = 100
theta_min, theta_max = 1, 2

In [0]:
theta = np.linspace(theta_min, theta_max, N)
dist = (1/N)*np.ones(N)

In [0]:
data = np.array([
    2.0,
    0.3,
    0.05
])

In [0]:
def mean(x):
    return np.sum(x*dist)
def std(x):
    return np.sum((x**2)*dist) - mean(x)**2
def q_func(alpha, rho, r, theta):
    return (alpha*r**(1-rho)+(1-alpha)*theta**(1-rho))**(1/(1-rho))
def con(alpha, rho, kappa, eta, zeta, r, qbar):
    q = q_func(alpha, rho, r, theta)
    dq = alpha*(q/r)**rho
    return [
        dq*qbar**zeta - kappa*r**eta,
        qbar - np.sum(q*dist)
    ]
def mmt(alpha, rho, kappa, eta, zeta, r, qbar):
    q = q_func(alpha, rho, r, theta)
    M = kappa*(r**(1+eta))/(1+kappa)
    prof = (qbar**zeta)*q
    return [
        mean(prof),
        std(prof),
        mean(M)
    ]
def obj(alpha, rho, kappa, eta, zeta, r, qbar):
    theory = np.array(mmt(alpha, rho, kappa, eta, zeta, r, qbar))
    return -np.sum(((theory-data)/data)**2)

In [0]:
vmin = {
    'eta': 0.0
}

In [0]:
var0 = {
    'alpha': 0.5,
    'rho': 1.5,
    'kappa': 0.01,
    'eta': 1.0,
    'zeta': 0.02,
    'r': 0.1*np.ones(N),
    'qbar': 1.0
}

In [0]:
%time var1 = grad_constr_jax.constrained_gradient_descent(obj, con, var0, vmin=vmin, output=True)

In [0]:
var1

## Micro Model

In [0]:
N = 64 # number of grid points
theta_min, theta_max = 1, 2

In [0]:
theta = np0.linspace(theta_min, theta_max, N)
pmf = (1/N)*np.ones(N)
cmf = np.cumsum(pmf)

In [0]:
def q_func(alpha, rho, r, theta):
    return (alpha*r**(1-rho)+(1-alpha)*theta**(1-rho))**(1/(1-rho))
def equil(alpha, rho, kappa, eta, zeta, r, qbar):
    q = q_func(alpha, rho, r, theta)
    dq = alpha*(q/r)**rho
    return [
        dq*qbar**zeta - kappa*r**eta,
        qbar - np.sum(q*pmf)
    ]

In [0]:
def solve_model(alpha, rho, kappa, eta, zeta, **kwargs):
    con = lambda r, qbar: equil(alpha, rho, kappa, eta, zeta, r, qbar)
    var0 = {
        'r': 0.1*np.ones(N),
        'qbar': 1.0
    }
    return gc.gradient_lstsq(con, var0, **kwargs)

In [0]:
par0 = {
    'alpha': 0.3,
    'rho': 1.5,
    'kappa': 0.1,
    'eta': 2.0,
    'zeta': 0.02
}

In [0]:
sol = solve_model(**par0, vmin={'qbar': 0.01}, step=0.1, output=True)
print(sol['qbar'], np.mean(sol['r']))

In [0]:
M = 128 # number of firms in data

In [0]:
data = pd.DataFrame({
    'bin': nd.random_vec(cmf, M)
})
data['theta'] = theta[data['bin']]
data['r'] = sol['r'][data['bin']]
data['q'] = q_func(par0['alpha'], par0['rho'], data['r'], data['theta'])
data.head()

In [0]:
def mean(x):
    return np.sum(x*dist)
def std(x):
    return np.sum((x**2)*dist) - mean(x)**2
def mmt(alpha, rho, kappa, eta, zeta, r, qbar):
    q = q_func(alpha, rho, r, theta)
    M = kappa*(r**(1+eta))/(1+kappa)
    prof = (qbar**zeta)*q
    return [
        mean(prof),
        std(prof),
        mean(M)
    ]
def obj(alpha, rho, kappa, eta, zeta, r, qbar):
    theory = np.array(mmt(alpha, rho, kappa, eta, zeta, r, qbar))
    return -np.sum(((theory-data)/data)**2)

In [0]:
vmin = {
    'eta': 0.0
}

In [0]:
var0 = {
    'alpha': 0.5,
    'rho': 1.5,
    'kappa': 0.01,
    'eta': 1.0,
    'zeta': 0.02,
    'r': 0.1*np.ones(N),
    'qbar': 1.0
}

In [0]:
%time var1 = grad_constr_jax.constrained_gradient_descent(obj, equil, var0, vmin=vmin, output=True)

In [0]:
var1